# 第一章 理解高性能 Python

## 基本的计算机系统
一台计算机的底层组件可被分为三大基本部分:
* 计算单元:计算单元有一个属性告诉我们它每秒能够进行多少次计算
* 存储单元:存储单元有一个属性告诉我们它能保存多少数据,还有一个属性告诉我们能以多快的速度对它进行读写
* 以及两者之间的连接:而连接则有一个属性告诉我们它们能以多快的速度将数据从一个地方移动到另一个地方。



## 通信层

一条总线的主要属性是它的速度:在给定时间内它能传输多少数据。该属性由两个因素决定
* 一次能传输多少数据(总线带宽)和每秒能传输几次(总线频率)。
需要说明的是一次传输的数据总是有序的:一块数据先从内存中读出,然后被移动到另一个地方。
这就是为什么总线的速度可以被拆分为两个因素,因为这两个因素分别独立影响计算的不同方面:
高的总线带宽可以一次性移动所有相关数据,有助于矢量化的代码(或任何顺序读取内存的代码),
而另一方面,低带宽高频率有助于那些经常随机读取内存的代码。
有意思的是,这些属性是由计算机设计者在主板的物理布局上决定的:
* 当芯片之间相距较近时,它们之间的物理链路就较短,就可以允许更高的传输速度。而物理链路的数量则决定了总线的带宽(带宽这个词真的具有物理上的意义!)。

## 理想计算模型和 Python 虚拟机

* 首先是 Python 对象不再是内存中最优化的布局。这是因为Python 是一种垃圾收集语言——内存会被自动分配并在需要时释放。这会导致内存碎片并影响向 CPU 缓存的传输。
* 根源是 Python 的动态类型以及 Python 并不是一门编译性的语言。很多 C 语言开发者已经在多年开发过程中意识到,编译器总是比你聪明。
* 由于 GIL,一次仅有一个核心可以被使用.我们可以使用多进程(multiprocessing 模块)而不是多线程,或者使用 Cython 或外部函数来避免这个问题

# 第二章 通过性能分析找到瓶颈

## 高效地分析性能
* 性能分析的首要目标是对受测系统进行测试来发现哪里太慢(或占用太多 RAM, 或导致太多磁盘 I/O 或网络 I/O)。性能分析一般会导致额外的性能开销

## 计时的简单方法——打印和修饰

In [10]:
%time

from functools import wraps
import time

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        results= fn(*args, **kwargs)
        print("total:",time.time()-t1)
        return results
    return measure_time

@timefn
def my_mul():
    x= [index**3 for index in range(3000000)]
    return x


zz = my_mul()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs
total: 1.0813963413238525


## 用 UNIX 的 time 命令进行简单的计时

* 用 UNIX 的 time 命令进行简单的计时

```
/usr/bin/time -p python your_python.py
```
* 打开--verbose 开关来获得更多输出信息
```
/usr/bin/time --verbose python your_python.py
```

## 使用 cProfile 模块
-s cumulative 开关告诉 cProfile 对每个函数累计花费的时间进行排序,这能让我们看到代码最慢的部分
```
python -m cProfile -s cumulative julia1_nopil.py
```


## 用 runsnakerun 对 cProfile 的输出进行可视化
* runsnake 是一个可视化工具,用于显示 cProfile 创建的统计文件—你只需要看它生成的图像就可以快速意识到哪个函数开销最大

## 用 line_profiler 进行逐行分析
* 输入命令 pip install line_profiler 来安装 line_profiler。用修饰器(@profile)标记选中的函数。用 kernprof.py 脚本运行你的代码,被选函数每一行花费的 CPU 时间以及其他信息就会被记录下来
* line_profiler用来测量 CPU 占用率

## 用 memory_profiler 诊断内存的用量
```
pip install memory_profiler
python -m memory_profiler julia1_memoryprofiler.py
```
* 能够逐行测量内存(RAM)占用率

mprof可视化
```
mprof run memory_profiler_test.py 
mprof plot 
mprof clean
```

## 用 heapy 调查堆上的对象
* 为了使用 heapy,你需要用命令 pip install guppy 安装 guppy 包

## 用 dowser 实时画出变量的实例

## 用 dis 模块检查 CPython 字节码

In [12]:
import dis
dis.dis(my_mul)

  9           0 LOAD_GLOBAL              0 (time)
              2 LOAD_METHOD              0 (time)
              4 CALL_METHOD              0
              6 STORE_FAST               2 (t1)

 10           8 LOAD_DEREF               0 (fn)
             10 LOAD_FAST                0 (args)
             12 LOAD_FAST                1 (kwargs)
             14 CALL_FUNCTION_EX         1
             16 STORE_FAST               3 (results)

 11          18 LOAD_GLOBAL              1 (print)
             20 LOAD_CONST               1 ('total:')
             22 LOAD_GLOBAL              0 (time)
             24 LOAD_METHOD              0 (time)
             26 CALL_METHOD              0
             28 LOAD_FAST                2 (t1)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            2
             34 POP_TOP

 12          36 LOAD_FAST                3 (results)
             38 RETURN_VALUE


## 不同的方法不同的复杂度

In [14]:
def fn_expressive(upper = 1000000):
    total = 0
    for n in range(upper):
        total += n
    return total
def fn_terse(upper = 1000000):
    return sum(range(upper))
%timeit fn_expressive()

66.6 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit fn_terse()

21.3 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
import dis
dis.dis(fn_expressive)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (total)

  3           4 SETUP_LOOP              24 (to 30)
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_FAST                0 (upper)
             10 CALL_FUNCTION            1
             12 GET_ITER
        >>   14 FOR_ITER                12 (to 28)
             16 STORE_FAST               2 (n)

  4          18 LOAD_FAST                1 (total)
             20 LOAD_FAST                2 (n)
             22 INPLACE_ADD
             24 STORE_FAST               1 (total)
             26 JUMP_ABSOLUTE           14
        >>   28 POP_BLOCK

  5     >>   30 LOAD_FAST                1 (total)
             32 RETURN_VALUE


In [19]:
dis.dis(fn_terse)

  7           0 LOAD_GLOBAL              0 (sum)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_FAST                0 (upper)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 RETURN_VALUE


## **在优化期间进行单元测试保持代码的正确性**